# Ground state calculations of the double well quantum dot

Given a basis set $\{\psi_p\}_{p = 1}^L$ of single particle functions that are eigenfunctions to the one-body Hamiltonian operator for the double well system, $h_{\text{DW}}$, i.e.,

\begin{align}
    h_{\text{DW}}\rvert\psi_p\rangle = \epsilon_p\rvert\psi_p\rangle,
\end{align}

where $\epsilon_p$ is the eigenenergy of the single particle function $\rvert\psi_p\rangle$. The double well one-body operator (with a perturbation in the $x$-direction) is given by

\begin{align}
    h_{\text{DW}}
    &= -\frac{\hbar^2}{2m}\nabla^2 + \frac{1}{2}m\omega^2 r^2
        + \frac{1}{2}m\omega^2\left(
            \frac{1}{4}R^2 - R|x|
        \right) \\
    &= h_{\text{HO}} + \frac{1}{2}m\omega^2\left(
            \frac{1}{4}R^2 - R|x|
        \right),
\end{align}
where $h_{\text{HO}}$ is the one-body Hamiltonian for the harmonic oscillator. We can solve this directly by discretizing the space we are working in and constructing the full matrix $h_{\text{DW}}$ before diagonalizing.

## Solving the one-dimensional double well quantum dot directly

By limiting ourselves to one dimension we can set up the one-body Hamiltonian matrix directly and diagonalize it to find the eigenenergies, $\epsilon_p$, and the corresponding eigenfunctions, $\rvert\psi_p\rangle$.